# Create a time-of-flight lookup table for BIFROST

In [ ]:
from ess.reduce import nexus
import sciline
import scipp as sc
import scippnexus as snx
from ess.reduce.nexus.types import RawChoppers, DiskChoppers
from ess.reduce.time_of_flight.lut import LtotalRange, NumberOfSimulatedNeutrons, SourcePosition, TofLookupTableWorkflow
from scippneutron.chopper import DiskChopper

from ess.bifrost import BifrostSimulationWorkflow
from ess.bifrost.data import simulated_elastic_incoherent_with_phonon
from ess.spectroscopy.types import *

## Load and process beamline parameters

First, load all required beamline parameters from an input NeXus file.
We only need to know the geometry up to the sample which is the same for all banks, so choosing a single detector is enough.

In [ ]:
input_filename = simulated_elastic_incoherent_with_phonon()
with snx.File(input_filename) as f:
    detector_names = list(f['entry/instrument'][snx.NXdetector])

In [ ]:
bifrost_workflow = BifrostSimulationWorkflow(detector_names)
bifrost_workflow[Filename[SampleRun]] = input_filename

M = nexus.types.CalibratedMonitor[SampleRun, FrameMonitor3]
C = RawChoppers[SampleRun]
choppers, monitor = bifrost_workflow.compute((C, M)).values()
beamline = sciline.compute_mapped(bifrost_workflow, BeamlineWithSpectrometerCoords[SampleRun])[0]

Compute the required distance range to include the monitor and sample:

In [ ]:
l_monitor = sc.norm(monitor.coords['source_position'] - monitor.coords['position'])
l_min = l_monitor
l_max = beamline.coords['L1']

The choppers in the simulated file need to be processed before they can be used for computing a lookup table.
The following works for the specific simulation but is **not** usable in general.

In [ ]:
def extract_chopper_plateau(chopper):
    processed = chopper.copy()
    # These are constant in the simulated data.
    processed['rotation_speed'] = processed['rotation_speed'].data.mean()
    processed['phase'] = processed['phase'].data.mean()
    # Guessing here as this is not stored in the file.
    processed['beam_position'] = sc.scalar(0.0, unit='deg')
    return DiskChopper.from_nexus(processed)


disk_choppers = choppers.apply(extract_chopper_plateau)

## Compute the lookup table

Construct a lookup table workflow:

In [ ]:
workflow = TofLookupTableWorkflow()
workflow[DiskChoppers] = disk_choppers
workflow[LtotalRange] = (l_min, l_max)
workflow[NumberOfSimulatedNeutrons] = 5_000_000
workflow[SourcePosition] = beamline.coords['source_position']

In [ ]:
workflow.visualize(TimeOfFlightLookupTable, graph_attr={"rankdir": "LR"})

Compute a lookup table:

In [ ]:
table = workflow.compute(TimeOfFlightLookupTable)
table

In [ ]:
table.squeeze().plot()

## Save to file

In [ ]:
table.save_hdf5('BIFROST-simulation-tof-lookup-table.h5')